In [1]:
import sys
sys.path.append('/Users/victoria/Documents/Scripts/Python/DGL-PTM/dgl_ptm')
import dgl_ptm
import os
os.environ["DGLBACKEND"] = "pytorch"

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
model = dgl_ptm.PovertyTrapModel(model_identifier='default')

In [3]:
model.set_model_parameters(default=True)
print(model.steering_parameters)

{'npath': './agent_data.zarr', 'epath': './edge_data', 'ndata': ['all_except', ['a_table']], 'edata': ['all'], 'mode': 'xarray', 'wealth_method': 'singular_transfer', 'income_method': 'pseudo_income_generation', 'tech_gamma': tensor([0.3000, 0.3500, 0.4500]), 'tech_cost': tensor([0.0000, 0.1500, 0.6500]), 'consume_method': 'pseudo_consumption', 'adapt_m': tensor([0.0000, 0.5000, 0.9000]), 'adapt_cost': tensor([0.0000, 0.2500, 0.4500]), 'depreciation': 0.6, 'discount': 0.95, 'm_theta_dist': {'type': 'multinomial', 'parameters': [[0.01, 0.1, 0.79, 0.1], [0.1, 0.5, 0.8, 1]], 'round': False, 'decimals': None}, 'del_prob': 0.05, 'ratio': 0.1, 'weight_a': 0.69, 'weight_b': 35, 'truncation_weight': 1e-10, 'step_type': 'default'}


In [4]:
model.initialize_model()

In [5]:
model.run()

performing step 0 of 20
creating bidirectional link between nodes 1 (src) and tensor([20]) (dst)
created 1 of 1 links requested
performing step 1 of 20
creating bidirectional link between nodes 42 (src) and tensor([29]) (dst)
created 1 of 1 links requested
performing step 2 of 20
dst node 84 is an end node
no FoF link possible for src dst nodes 1 and 84.
created 0 of 1 links requested
performing step 3 of 20
creating bidirectional link between nodes 48 (src) and tensor([92]) (dst)
created 1 of 1 links requested
performing step 4 of 20
creating bidirectional link between nodes 11 (src) and tensor([23]) (dst)
created 1 of 1 links requested
performing step 5 of 20
creating bidirectional link between nodes 13 (src) and tensor([20]) (dst)
created 1 of 1 links requested
performing step 6 of 20
dst node 86 is an end node
no FoF link possible for src dst nodes 58 and 86.
created 0 of 1 links requested
performing step 7 of 20
creating bidirectional link between nodes 75 (src) and tensor([56]) (

In [6]:
model = dgl_ptm.PovertyTrapModel(model_identifier='test')

In [9]:
import torch
model.set_model_parameters(default=False, **{'steering_parameters': {'npath':'./agent_data.zarr',
                            'epath':'./edge_data', 
                            'ndata':['all_except',['a_table']],
                            'edata':['all'],
                            'mode':'xarray',
                            'wealth_method':'singular_transfer',
                            'income_method':'income_generation',
                            'tech_gamma': torch.tensor([0.3,0.35,0.45]),
                            'tech_cost': torch.tensor([0,0.15,0.65]),
                            'consume_method':'optimized_consumption',
                            'adapt_m':torch.tensor([0,0.5,0.9]),
                            'adapt_cost':torch.tensor([0,0.25,0.45]),
                            'depreciation': 0.6,
                            'discount': 0.95,
                            'm_theta_dist': {'type':'multinomial','parameters':[[0.01 ,0.1, 0.79, 0.1],[0.1, 0.5, 0.8, 1]],'round':False,'decimals':None},
                            'del_prob':0.05,
                            'ratio':0.1,
                            'weight_a':0.69,
                            'weight_b':35, 
                            'truncation_weight':1.0e-10,
                            'step_type': 'custom'}})

print(model.number_agents)  
print(model.steering_parameters)                        

100
{'npath': './agent_data.zarr', 'epath': './edge_data', 'ndata': ['all'], 'edata': ['all'], 'mode': 'xarray', 'wealth_method': 'singular_transfer', 'income_method': 'pseudo_income_generation', 'tech_gamma': tensor([0.3000, 0.3500, 0.4500]), 'tech_cost': tensor([0.0000, 0.1500, 0.6500]), 'consume_method': 'pseudo_consumption', 'depreciation': 0.6, 'discount': 0.95, 'del_prob': 0.05, 'ratio': 0.1, 'weight_a': 0.69, 'weight_b': 35, 'truncation_weight': 1e-10, 'step_type': 'custom'}


In [10]:
model.initialize_model()

KeyError: 'adapt_m'

In [ ]:
import torch
import numpy as np
TechTable =  np.array([[0.3,0],[0.35,0.15],[0.45, 0.65]])
wealth=torch.tensor([ 6.4072, 17.6514,  8.5695, 27.9308, 13.7166])
alpha=torch.tensor([1.0997, 0.9967, 1.0678, 0.9097, 1.0579])
gamma = torch.tensor([0.3,0.35,0.45])
cost = torch.tensor([0,0.15,0.65])

income,index = torch.max((alpha[:,None]*wealth[:,None]**TechTable[:,0] - TechTable[:,1]), axis=1)
income=income.to(torch.float32)

print(income)
print(index)


income,tech_index = torch.max((alpha[:,None]*wealth[:,None]**gamma - cost), axis=1)

print(income.dtype)
print(index)

tensor([1.9567, 2.9776, 2.1575, 3.4204, 2.7872])
tensor([1, 2, 2, 2, 2])
torch.float32
tensor([1, 2, 2, 2, 2])


In [ ]:
def sample_distribution_tensor(type, distParameters, nSamples, round=False, decimals=None):
    """
    create and return samples from different distributions

    :param type: Type of distribution to sample
    :param distParameters: array of parameters as required/supported by requested distribution type
    :param nSamples: number of samples to return (as 1d tensor)
    :param round: optional, whether the samples are to be rounded
    :param decimals: optional, required if round is specified. decimal places to round to
    """
    if type == 'uniform':
        dist = torch.distributions.uniform.Uniform(torch.tensor(distParameters[0]),torch.tensor(distParameters[1])).sample(torch.tensor([nSamples]))
    elif type == 'normal':
        dist = torch.distributions.normal.Normal(torch.tensor(distParameters[0]),torch.tensor(distParameters[1])).sample(torch.tensor([nSamples]))
    elif type == 'bernoulli':
        dist = torch.distributions.bernoulli.Bernoulli(probs=distParameters[0],logits=distParameters[1],validate_args=None).sample(torch.tensor([nSamples]))
    elif type == 'multinomial':
        dist = torch.gather(torch.Tensor(distParameters[1]), 0, torch.multinomial(torch.tensor(distParameters[0]), nSamples, replacement=True))

    else:
        raise NotImplementedError('Currently only uniform, normal, multinomial, and bernoulli distributions are supported')

    if round:
        if decimals == None:
            raise ValueError('rounding requires decimals of rounding accuracy to be specified')
        else:
            return torch.round(dist,decimals=decimals)
    else:
        return dist

theta_dist = {'type':'multinomial','parameters':[[0.01 ,0.1, 0.79, 0.1],[0.1, 0.5, 0.8, 1]],'round':False,'decimals':None} 
theta_vals = torch.tensor([0.1, 0.5, 0.8, 1])
capital_dist = {'type':'uniform','parameters':[0.1,10.],'round':False,'decimals':None}


agentsCapital = sample_distribution_tensor(capital_dist['type'],capital_dist['parameters'],10,capital_dist['round'],decimals=capital_dist['decimals'])
print(agentsCapital)
modelTheta = sample_distribution_tensor(theta_dist['type'],theta_dist['parameters'],10,round=theta_dist['round'],decimals=theta_dist['decimals'])
print(modelTheta[1])


tensor([9.0460, 5.8420, 4.0483, 2.4734, 7.5922, 3.3757, 0.2560, 9.5808, 8.8816,
        7.0898])
tensor(0.5000)


In [ ]:
tech_gamma=torch.tensor([0.3,0.35,0.45])
tech_cost=torch.tensor([0,0.15,0.65])
α=1
k=2



f = []
for i in range(tech_gamma.size(dim=0)): 
    #in the end, they may need their own tech tables
    entry = α * k**tech_gamma[i] - tech_cost[i]
    f.append(entry)
print(f)
entry = α * k**tech_gamma - tech_cost
print(entry)


[tensor(1.2311), tensor(1.1246), tensor(0.7160)]
tensor([1.2311, 1.1246, 0.7160])


In [ ]:
torch.stack([tech_gamma,tech_cost]).repeat(4,1,1)


tensor([[[0.3000, 0.3500, 0.4500],
         [0.0000, 0.1500, 0.6500]],

        [[0.3000, 0.3500, 0.4500],
         [0.0000, 0.1500, 0.6500]],

        [[0.3000, 0.3500, 0.4500],
         [0.0000, 0.1500, 0.6500]],

        [[0.3000, 0.3500, 0.4500],
         [0.0000, 0.1500, 0.6500]]])

In [13]:
data={'a':0,'b':4,'c':12}
params={'steering_parameters':{'m_theta_dist': {'type':'multinomial','parameters':[[0.01 ,0.1, 0.79, 0.1],[0.1, 0.5, 0.8, 1]],'round':False,'decimals':None},
                            'del_prob':0.05},'ndtata':['all']}

print(params['steering_parameters']['m_theta_dist']['parameters'][1])


def printdata(ndata):
    if ndata == ['all']:
        ndata = list(data.keys())

    if ndata[0] == 'allexcept':
        ndata = list(data.keys() - ndata[1])
    print(ndata)


printdata(['all'])
printdata(['allexcept',['a']])
printdata(['allexcept',['a','c']])

[0.1, 0.5, 0.8, 1]
['a', 'b', 'c']
['b', 'c']
['b']
